In [111]:
import pandas as pd
#Import Packages
import ase
import os
from ase.io import read
import numpy as np
import csv
from ase.io.jsonio import read_json
import json
from scipy.stats import rankdata
from ase.visualize import view
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn.functional as F
from torch_geometric.data import Dataset, Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import dense_to_sparse, degree, add_self_loops
import torch_geometric.transforms as T
import torch_geometric.utils

import glob, os

import networkx as nx

import trimesh
import pickle


# x = pd.read_pickle("ZeoGraphs.p")
# for i in x:
#     y = torch_geometric.utils.from_networkx(i)
#     break

In [144]:
import os.path as osp

import torch


class ZeoliteDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super(ZeoliteDataset, self).__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['ZeoGraphs.p', 'ZeoUnitCells.']

    @property
    def processed_file_names(self):
        return "not-implemented.pt"
    
    def download(self):
        pass

    def makeAdj(self, graphs, num_pad):
        adj = torch.zeros(len(graphs), num_pad, num_pad)
        for i in range(len(graphs)):
            graph_tmp = torch.from_numpy(nx.to_numpy_array(graphs[i]))
            adj[i, :, :] = F.pad(graph_tmp, (0, num_pad - graph_tmp.shape[1], 0, num_pad - graph_tmp.shape[0]), "constant", 0)

    def process(self):
        idx = 0
        self.graphs = pd.read_pickle(self.raw_paths[0])

        if  not os.listdir(self.processed_dir):
            for graph in self.graphs:
                # Read data from `raw_path
                adj = torch.zeros(432, 432)

                graph_tmp = torch.from_numpy(nx.to_numpy_array(graph))
                adj = np.pad(graph_tmp, ((0, 432 - graph_tmp.shape[1]), (0, 432 - graph_tmp.shape[0])), "constant",constant_values = (0,0))
                adj = torch.from_numpy(adj)


                if self.pre_filter is not None and not self.pre_filter(data):
                    continue

                if self.pre_transform is not None:
                    data = self.pre_transform(data)

                torch.save(adj, osp.join(self.processed_dir, f'data_{idx}.pt'))
                idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data
    



In [146]:
dataset = ZeoliteDataset("Repeated_Cells_1x")

Processing...
Done!


In [155]:
loader.dataset.get(1)

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [153]:
loader = DataLoader(dataset,batch_size=64,shuffle=True)

In [154]:
for data in loader:
 print(data)
 break

TypeError: DataLoader found invalid type: <class 'numpy.ndarray'>